# LangSmith Prompt Versioning 빠른 예제
LangSmith 허브에서 프롬프트를 저장하고 버전을 관리하는 최소한의 흐름입니다.

## 1. 사전 준비
필요한 패키지를 불러오고 `.env`에서 API 키를 로드한 뒤 LangSmith 추적을 켭니다.

In [1]:
pip install langsmith langchain langchain-openai pandas


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from langchain_teddynote import logging
from langsmith import Client

load_dotenv()
logging.langsmith("LangSmith-versioning-demo")

client = Client()
PROMPT_ID = "prompt_rag"  # 필요에 맞게 수정하세요
PROMPT_REPO = PROMPT_ID if "/" in PROMPT_ID else f"-/{PROMPT_ID}"

ModuleNotFoundError: No module named 'dotenv'

## 2. 프롬프트 최초 등록 (버전 1)
시스템/유저 메시지를 정의하고 `push_prompt`로 LangSmith에 업로드합니다.

In [1]:
from langchain_core.prompts import ChatPromptTemplate

base_system_prompt = """
You answer using only the given CONTEXT. If 정보가 없으면 모른다고 말하세요.
"""

user_prompt_template = """
## CONTEXT ##
{context}
## QUESTION ##
{question}
"""

prompt_v1 = ChatPromptTemplate.from_messages([
    ("system", base_system_prompt.strip()),
    ("user", user_prompt_template.strip()),
])

prompt_v1_url = client.push_prompt(
    PROMPT_ID,
    object=prompt_v1,
    description="컨텍스트 QA 프롬프트 v1",
    tags=["demo", "v1"],
)
print("v1 등록 URL:", prompt_v1_url)

ModuleNotFoundError: No module named 'langchain_core'

## 3. 프롬프트 수정 후 새 버전 업로드
답변 형식을 더 구조화했다고 가정하고 다시 `push_prompt`를 호출합니다.

In [11]:
improved_system_prompt = """
You answer using only the given CONTEXT.
- 첫 줄에 한 문장 요약을 작성하세요.
- 이어서 최대 3개의 bullet로 근거를 정리하세요.
- 답을 찾을 수 없으면 모른다고 말하세요.
"""

prompt_v2 = ChatPromptTemplate.from_messages([
    ("system", improved_system_prompt.strip()),
    ("user", user_prompt_template.strip()),
])

prompt_v2_url = client.push_prompt(
    PROMPT_ID,
    object=prompt_v2,
    description="요약과 bullet을 추가한 v2",
    tags=["demo", "v2"],
)
print("v2 등록 URL:", prompt_v2_url)

v2 등록 URL: https://smith.langchain.com/prompts/prompt_rag/1bca06e8?organizationId=be958e69-b72a-4544-b202-0eadd9569706


## 4. 버전 확인 및 특정 커밋 불러오기
`list_prompt_commits`로 히스토리를 보고, 커밋 해시를 이용해 과거 버전을 불러옵니다.

In [ ]:
# 기본 사용 법 : 관련 커밋으로 pull 받아오기 가능 
client.pull_prompt("prompt_rag")
client.pull_prompt("prompt_rag:5e12e879")

In [12]:
from datetime import timezone

commits = list(client.list_prompt_commits(PROMPT_REPO, limit=3))
for idx, commit in enumerate(commits, start=1):
    created = commit.created_at
    if created and created.tzinfo:
        created = created.astimezone(timezone.utc)
    timestamp = created.strftime("%Y-%m-%d %H:%M:%S UTC") if created else "Unknown"
    print(f"{idx}. {commit.commit_hash[:8]} | {timestamp}")

if commits:
    latest_prompt = client.pull_prompt(PROMPT_ID)
    print("최신 버전 type:", type(latest_prompt))

if len(commits) >= 2:
    previous_hash = commits[1].commit_hash
    previous_prompt = client.pull_prompt(f"{PROMPT_ID}:{previous_hash}")
    print("이전 버전 해시:", previous_hash[:8])
    print("이전 버전 type:", type(previous_prompt))
else:
    print("이전 버전이 없다면 push_prompt를 한 번 더 실행해 보세요.")

1. 1bca06e8 | 2025-10-31 06:09:29 UTC
2. 05b5f416 | 2025-10-31 06:08:47 UTC
3. 56e69a64 | 2025-10-31 05:53:29 UTC
최신 버전 type: <class 'langchain_core.prompts.chat.ChatPromptTemplate'>
이전 버전 해시: 05b5f416
이전 버전 type: <class 'langchain_core.prompts.chat.ChatPromptTemplate'>
